# Logistic Regression Consulting Project

## Binary Customer Churn

A marketing agency has many customers that use their service to produce ads for the client/customer websites. They've noticed that they have quite a bit of churn in clients. They basically randomly assign account managers right now, but want you to create a machine learning model that will help predict which customers will churn (stop buying their service) so that they can correctly assign the customers most at risk to churn an account manager. Luckily they have some historical data, can you help them out? Create a classification algorithm that will help classify whether or not a customer churned. Then the company can test this against incoming data for future customers to predict which customers will churn and assign them an account manager.

The data is saved as customer_churn.csv. Here are the fields and their definitions:

    Name : Name of the latest contact at Company
    Age: Customer Age
    Total_Purchase: Total Ads Purchased
    Account_Manager: Binary 0=No manager, 1= Account manager assigned
    Years: Totaly Years as a customer
    Num_sites: Number of websites that use the service.
    Onboard_date: Date that the name of the latest contact was onboarded
    Location: Client HQ Address
    Company: Name of Client Company
    
Once you've created the model and evaluated it, test out the model on some new data (you can think of this almost like a hold-out set) that your client has provided, saved under new_customers.csv. The client wants to know which customers are most likely to churn given this data (they don't have the label yet).

# Load Data and Libraries 

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("logReg").getOrCreate()

In [3]:
from pyspark.ml.classification import LogisticRegression

In [5]:
my_data = spark.read.csv("../../Data/customer_churn.csv",inferSchema=True, header=True)

In [6]:
my_data.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [11]:
my_data.describe().show() 
# we see no mssing data 

+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+-------------------+--------------------+--------------------+-------------------+
|summary|        Names|              Age|   Total_Purchase|   Account_Manager|            Years|         Num_Sites|       Onboard_date|            Location|             Company|              Churn|
+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+-------------------+--------------------+--------------------+-------------------+
|  count|          900|              900|              900|               900|              900|               900|                900|                 900|                 900|                900|
|   mean|         null|41.81666666666667|10062.82403333334|0.4811111111111111| 5.27315555555555| 8.587777777777777|               null|                null|                null|0.16666666666666666|
| stddev| 

# Processing 

In [16]:
# select some colums 
from pyspark.ml.feature import VectorAssembler 
assembler = VectorAssembler(inputCols= ["Age","Total_Purchase","Years","Num_Sites"],outputCol="features")

In [19]:
output = assembler.transform(my_data)

In [23]:
my_final_data_set = output.select(["features","churn"])
train_data, test_data = my_final_data_set.randomSplit([0.7,0.3])

In [24]:
from pyspark.ml.classification import LogisticRegression

In [29]:
lr = LogisticRegression(labelCol="churn")
lgr_model = lr.fit(train_data)
training_sum = lgr_model.summary

In [32]:
training_sum.predictions.describe().show()

+-------+------------------+------------------+
|summary|             churn|        prediction|
+-------+------------------+------------------+
|  count|               640|               640|
|   mean|         0.1671875|         0.1265625|
| stddev|0.3734350496495064|0.3327422870292634|
|    min|               0.0|               0.0|
|    max|               1.0|               1.0|
+-------+------------------+------------------+



# Evaluation Matric 

In [33]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator


In [34]:
pred = lgr_model.evaluate(test_data)

In [38]:
pred.predictions.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|churn|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[26.0,8787.39,5.4...|    1|[0.87711176840478...|[0.70622335122758...|       0.0|
|[26.0,8939.61,4.5...|    0|[5.97296807803750...|[0.99745979759021...|       0.0|
|[28.0,8670.98,3.9...|    0|[7.37339049690512...|[0.99937265757635...|       0.0|
|[29.0,9378.24,4.9...|    0|[4.41940723403472...|[0.98810190149846...|       0.0|
|[29.0,9617.59,5.4...|    0|[4.03897110829845...|[0.98268934965143...|       0.0|
|[29.0,13240.01,4....|    0|[6.80559796522720...|[0.99889366763483...|       0.0|
|[29.0,13255.05,4....|    0|[4.34204672043667...|[0.98715720971368...|       0.0|
|[30.0,6744.87,5.1...|    0|[3.17070791727916...|[0.95971696179086...|       0.0|
|[30.0,7960.64,2.7...|    1|[3.62627696788569...|[0.97407490957604...|       0.0|
|[30.0,8403.78,4

# check the Erea under the Curve ROC 

In [40]:
Chrun_eval = BinaryClassificationEvaluator(labelCol="churn",rawPredictionCol="prediction")

In [87]:
auc = Chrun_eval.evaluate (pred.predictions)
auc

0.674525774300718

In [88]:
my_final_data_set.printSchema()

root
 |-- features: vector (nullable = true)
 |-- churn: integer (nullable = true)



# predict on new data

In [100]:
final_lr_model = lr.fit(my_final_data_set)

In [107]:
new_custormers = spark.read.csv("../../Data/new_customers.csv", header=True, inferSchema=True)

In [112]:
new_custormers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- features: vector (nullable = true)



In [109]:
new_custormers= assembler.transform(new_custormers)

In [113]:
new_custormers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- features: vector (nullable = true)



In [118]:
new_costu = new_custormers.select(["features"])
model = final_lr_model.transform(new_costu)

In [122]:
model.printSchema()

root
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



In [121]:
model.select("prediction").show()

+----------+
|prediction|
+----------+
|       0.0|
|       1.0|
|       1.0|
|       1.0|
|       0.0|
|       1.0|
+----------+

